In [1]:
from imblearn.combine import SMOTETomek
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
import numpy as np
import cv2
import os
from collections import Counter

In [2]:
# Chargement de la dataset
def load_images_and_labels(dataset_path, img_size=(224, 224)):
    images, labels = [], []
    for label_dir in os.listdir(dataset_path):
        for img_file in os.listdir(os.path.join(dataset_path, label_dir)):
            img_path = os.path.join(dataset_path, label_dir, img_file)
            image = cv2.imread(img_path)
            image = cv2.resize(image, img_size)
            images.append(image)
            labels.append(label_dir)
    return np.array(images), np.array(labels)




In [3]:
images, labels = load_images_and_labels("./wheat_leaf")
images = images / 255.0  # Normalisation des images


In [4]:
# Encodage des étiquettes
lb = LabelBinarizer()
labels = lb.fit_transform(labels)  # Encodage des étiquettes en one-hot


In [5]:
# Division en ensembles train, validation et test
X_train, X_temp, y_train, y_temp = train_test_split(images, labels, test_size=0.3, random_state=42)
X_valid, X_test, y_valid, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)


In [6]:
# Vérification de la distribution des classes dans le training set
class_distribution = Counter(np.argmax(y_train, axis=1))
print("Distribution des classes dans le training set :", class_distribution)


Distribution des classes dans le training set : Counter({2: 150, 1: 70, 0: 64})


In [7]:
# Application de SMOTETomek pour équilibrer les classes
X_train_flattened = X_train.reshape(X_train.shape[0], -1)  # Aplatir les images
smote_tomek = SMOTETomek(random_state=42)
X_resampled, y_resampled = smote_tomek.fit_resample(X_train_flattened, np.argmax(y_train, axis=1))


In [8]:
# Reshape des données augmentées
X_resampled = X_resampled.reshape(-1, 224, 224, 3)


In [9]:
from keras.utils import to_categorical

# Convertir y_resampled en one-hot encoding
y_resampled_one_hot = to_categorical(y_resampled, num_classes=labels.shape[1])

# Fusionner les données d'origine et les données augmentées
X_augmented_train = np.vstack((X_train, X_resampled))
y_augmented_train = np.vstack((y_train, y_resampled_one_hot))

print("Shape of X_augmented_train:", X_augmented_train.shape)
print("Shape of y_augmented_train:", y_augmented_train.shape)


Shape of X_augmented_train: (734, 224, 224, 3)
Shape of y_augmented_train: (734, 3)


### ______________________________________________________ Xception ______________________________________________

In [10]:
from tensorflow.keras.applications import Xception
# Construction du modèle MobileNetV2
Xception_model = Xception(input_shape=(224, 224, 3), weights="imagenet", include_top=False)
Xception_model.trainable = False

model = models.Sequential([
    Xception_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dense(labels.shape[1], activation='softmax')  # Nombre de classes dynamique
])

# Compilation du modèle
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

# Vérification des types de données
print("X_augmented_train dtype:", X_augmented_train.dtype)
print("y_augmented_train dtype:", y_augmented_train.dtype)


X_augmented_train dtype: float64
y_augmented_train dtype: float64


In [11]:
# Entraînement du modèle avec les données augmentées
model.fit(X_augmented_train, y_augmented_train, epochs=20, validation_data=(X_valid, y_valid))


Epoch 1/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 104s 3s/step - accuracy: 0.6739 - loss: 0.6462 - val_accuracy: 0.8689 - val_loss: 0.3130
Epoch 2/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 66s 3s/step - accuracy: 0.9630 - loss: 0.1222 - val_accuracy: 0.9180 - val_loss: 0.2184
Epoch 3/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 67s 3s/step - accuracy: 0.9899 - loss: 0.0577 - val_accuracy: 0.9180 - val_loss: 0.2297
Epoch 4/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 68s 3s/step - accuracy: 1.0000 - loss: 0.0338 - val_accuracy: 0.9016 - val_loss: 0.2227
Epoch 5/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 68s 3s/step - accuracy: 0.9970 - loss: 0.0273 - val_accuracy: 0.8852 - val_loss: 0.3265
Epoch 6/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 68s 3s/step - accuracy: 1.0000 - loss: 0.0151 - val_accuracy: 0.9016 - val_loss: 0.2699
Epoch 7/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 68s 3s/step - accuracy: 1.0000 - loss: 0.0091 - val_accuracy: 0.9016 - val_loss: 0.3017
Epoch 8/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 69s 3s/step - accuracy: 1.0000 - loss: 0.0075 - val_accuracy: 0.9180 - val_loss

In [12]:
# Évaluation sur l'ensemble test
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Précision sur l'ensemble test : {accuracy:.2f}")

2/2 ━━━━━━━━━━━━━━━━━━━━ 10s 3s/step - accuracy: 0.9788 - loss: 0.1260
Précision sur l'ensemble test : 0.98


In [13]:
# Prédictions
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_test_classes = np.argmax(y_test, axis=1)


2/2 ━━━━━━━━━━━━━━━━━━━━ 17s 8s/step


In [14]:
from sklearn.metrics import confusion_matrix, classification_report

# Matrice de confusion et rapport de classification
conf_matrix = confusion_matrix(y_test_classes, y_pred_classes)
print("Matrice de confusion :")
print(conf_matrix)

# Rapport de classification
target_names = [str(cls) for cls in lb.classes_]
print("Rapport de classification :")
print(classification_report(y_test_classes, y_pred_classes, target_names=target_names))


Matrice de confusion :
[[18  0  0]
 [ 0 13  0]
 [ 1  0 30]]
Rapport de classification :
              precision    recall  f1-score   support

     Healthy       0.95      1.00      0.97        18
    septoria       1.00      1.00      1.00        13
 stripe_rust       1.00      0.97      0.98        31

    accuracy                           0.98        62
   macro avg       0.98      0.99      0.99        62
weighted avg       0.98      0.98      0.98        62



### _____________________________________________________  MobileNetV2 __________________________________________

In [15]:
from tensorflow.keras.applications import MobileNetV2
# Construction du modèle MobileNetV2
MobileNetV2_model = MobileNetV2(input_shape=(224, 224, 3), weights="imagenet", include_top=False)
MobileNetV2_model.trainable = False

model = models.Sequential([
    MobileNetV2_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dense(labels.shape[1], activation='softmax')  # Nombre de classes dynamique
])

# Compilation du modèle
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

# Vérification des types de données
print("X_augmented_train dtype:", X_augmented_train.dtype)
print("y_augmented_train dtype:", y_augmented_train.dtype)


X_augmented_train dtype: float64
y_augmented_train dtype: float64


In [16]:
# Entraînement du modèle avec les données augmentées
model.fit(X_augmented_train, y_augmented_train, epochs=20, validation_data=(X_valid, y_valid))


Epoch 1/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 21s 614ms/step - accuracy: 0.6497 - loss: 0.8954 - val_accuracy: 0.8525 - val_loss: 0.3277
Epoch 2/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 12s 503ms/step - accuracy: 0.9663 - loss: 0.1005 - val_accuracy: 0.9344 - val_loss: 0.1883
Epoch 3/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 12s 507ms/step - accuracy: 1.0000 - loss: 0.0368 - val_accuracy: 0.9508 - val_loss: 0.1593
Epoch 4/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 12s 502ms/step - accuracy: 1.0000 - loss: 0.0164 - val_accuracy: 0.9508 - val_loss: 0.1446
Epoch 5/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 23s 572ms/step - accuracy: 1.0000 - loss: 0.0110 - val_accuracy: 0.9508 - val_loss: 0.1384
Epoch 6/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 12s 514ms/step - accuracy: 1.0000 - loss: 0.0081 - val_accuracy: 0.9508 - val_loss: 0.1457
Epoch 7/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 12s 505ms/step - accuracy: 1.0000 - loss: 0.0058 - val_accuracy: 0.9508 - val_loss: 0.1501
Epoch 8/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 12s 505ms/step - accuracy: 1.0000 - loss: 0.0043 - val_accu

In [17]:
# Évaluation sur l'ensemble test
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Précision sur l'ensemble test : {accuracy:.2f}")


2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 1s/step - accuracy: 0.9362 - loss: 0.1194
Précision sur l'ensemble test : 0.94


In [18]:
# Prédictions
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_test_classes = np.argmax(y_test, axis=1)



2/2 ━━━━━━━━━━━━━━━━━━━━ 11s 6s/step


In [19]:
from sklearn.metrics import confusion_matrix, classification_report

# Matrice de confusion et rapport de classification
conf_matrix = confusion_matrix(y_test_classes, y_pred_classes)
print("Matrice de confusion :")
print(conf_matrix)

# Rapport de classification
target_names = [str(cls) for cls in lb.classes_]
print("Rapport de classification :")
print(classification_report(y_test_classes, y_pred_classes, target_names=target_names))


Matrice de confusion :
[[16  0  2]
 [ 0 12  1]
 [ 1  0 30]]
Rapport de classification :
              precision    recall  f1-score   support

     Healthy       0.94      0.89      0.91        18
    septoria       1.00      0.92      0.96        13
 stripe_rust       0.91      0.97      0.94        31

    accuracy                           0.94        62
   macro avg       0.95      0.93      0.94        62
weighted avg       0.94      0.94      0.94        62



### _______________________________________________ ResNet50V2 __________________________________________

In [20]:
from tensorflow.keras.applications import ResNet50V2
# Construction du modèle MobileNetV2
ResNet50V2_model = ResNet50V2(input_shape=(224, 224, 3), weights="imagenet", include_top=False)
ResNet50V2_model.trainable = False

model = models.Sequential([
    ResNet50V2_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dense(labels.shape[1], activation='softmax')  # Nombre de classes dynamique
])

# Compilation du modèle
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

# Vérification des types de données
print("X_augmented_train dtype:", X_augmented_train.dtype)
print("y_augmented_train dtype:", y_augmented_train.dtype)



X_augmented_train dtype: float64
y_augmented_train dtype: float64


In [21]:
# Entraînement du modèle avec les données augmentées
model.fit(X_augmented_train, y_augmented_train, epochs=20, validation_data=(X_valid, y_valid))


Epoch 1/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 83s 3s/step - accuracy: 0.6417 - loss: 0.8710 - val_accuracy: 0.8689 - val_loss: 0.2537
Epoch 2/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 52s 2s/step - accuracy: 0.9887 - loss: 0.0704 - val_accuracy: 0.8689 - val_loss: 0.2182
Epoch 3/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 83s 2s/step - accuracy: 0.9975 - loss: 0.0304 - val_accuracy: 0.8852 - val_loss: 0.1734
Epoch 4/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 81s 2s/step - accuracy: 1.0000 - loss: 0.0108 - val_accuracy: 0.8852 - val_loss: 0.1987
Epoch 5/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 48s 2s/step - accuracy: 1.0000 - loss: 0.0081 - val_accuracy: 0.8852 - val_loss: 0.2071
Epoch 6/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 49s 2s/step - accuracy: 1.0000 - loss: 0.0055 - val_accuracy: 0.8852 - val_loss: 0.2058
Epoch 7/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 48s 2s/step - accuracy: 1.0000 - loss: 0.0044 - val_accuracy: 0.8852 - val_loss: 0.2047
Epoch 8/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 50s 2s/step - accuracy: 1.0000 - loss: 0.0031 - val_accuracy: 0.8852 - val_loss:

In [23]:
# Évaluation sur l'ensemble test
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Précision sur l'ensemble test : {accuracy:.2f}")


2/2 ━━━━━━━━━━━━━━━━━━━━ 7s 2s/step - accuracy: 0.9362 - loss: 0.1118
Précision sur l'ensemble test : 0.94


In [24]:
# Prédictions
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_test_classes = np.argmax(y_test, axis=1)



1/2 ━━━━━━━━━━━━━━━━━━━━ 9s 10s/stepWARNING:tensorflow:6 out of the last 6 calls to <function TensorFlowTrainer.make_predict_function.<locals>.one_step_on_data_distributed at 0x000001F97EE7B100> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has reduce_retracing=True option that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/guide/function#controlling_retracing and https://www.tensorflow.org/api_docs/python/tf/function for  more details.
2/2 ━━━━━━━━━━━━━━━━━━━━ 18s 8s/step


In [25]:
from sklearn.metrics import confusion_matrix, classification_report

# Matrice de confusion et rapport de classification
conf_matrix = confusion_matrix(y_test_classes, y_pred_classes)
print("Matrice de confusion :")
print(conf_matrix)

# Rapport de classification
target_names = [str(cls) for cls in lb.classes_]
print("Rapport de classification :")
print(classification_report(y_test_classes, y_pred_classes, target_names=target_names))


Matrice de confusion :
[[16  0  2]
 [ 0 12  1]
 [ 1  0 30]]
Rapport de classification :
              precision    recall  f1-score   support

     Healthy       0.94      0.89      0.91        18
    septoria       1.00      0.92      0.96        13
 stripe_rust       0.91      0.97      0.94        31

    accuracy                           0.94        62
   macro avg       0.95      0.93      0.94        62
weighted avg       0.94      0.94      0.94        62



### ____________________________________________________ DenseNet121 __________________________________________

In [26]:
from tensorflow.keras.applications import DenseNet121
# Construction du modèle MobileNetV2
DenseNet121_model = DenseNet121(input_shape=(224, 224, 3), weights="imagenet", include_top=False)
DenseNet121_model.trainable = False

model = models.Sequential([
    DenseNet121_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dense(labels.shape[1], activation='softmax')  # Nombre de classes dynamique
])

# Compilation du modèle
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

# Vérification des types de données
print("X_augmented_train dtype:", X_augmented_train.dtype)
print("y_augmented_train dtype:", y_augmented_train.dtype)


X_augmented_train dtype: float64
y_augmented_train dtype: float64


In [27]:
# Entraînement du modèle avec les données augmentées
model.fit(X_augmented_train, y_augmented_train, epochs=20, validation_data=(X_valid, y_valid))


Epoch 1/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 108s 3s/step - accuracy: 0.6268 - loss: 0.7967 - val_accuracy: 0.8197 - val_loss: 0.3832
Epoch 2/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 50s 2s/step - accuracy: 0.9244 - loss: 0.2099 - val_accuracy: 0.9180 - val_loss: 0.1945
Epoch 3/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 50s 2s/step - accuracy: 0.9873 - loss: 0.0776 - val_accuracy: 0.9672 - val_loss: 0.1365
Epoch 4/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 52s 2s/step - accuracy: 0.9967 - loss: 0.0478 - val_accuracy: 0.9344 - val_loss: 0.1391
Epoch 5/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 53s 2s/step - accuracy: 1.0000 - loss: 0.0339 - val_accuracy: 0.9672 - val_loss: 0.1026
Epoch 6/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 50s 2s/step - accuracy: 1.0000 - loss: 0.0199 - val_accuracy: 0.9672 - val_loss: 0.0958
Epoch 7/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 85s 2s/step - accuracy: 1.0000 - loss: 0.0171 - val_accuracy: 0.9672 - val_loss: 0.0843
Epoch 8/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 51s 2s/step - accuracy: 1.0000 - loss: 0.0118 - val_accuracy: 0.9672 - val_loss

In [28]:

# Évaluation sur l'ensemble test
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Précision sur l'ensemble test : {accuracy:.2f}")


2/2 ━━━━━━━━━━━━━━━━━━━━ 6s 2s/step - accuracy: 0.9892 - loss: 0.0630
Précision sur l'ensemble test : 0.98


In [29]:

# Prédictions
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_test_classes = np.argmax(y_test, axis=1)


2/2 ━━━━━━━━━━━━━━━━━━━━ 33s 16s/step


In [30]:

from sklearn.metrics import confusion_matrix, classification_report

# Matrice de confusion et rapport de classification
conf_matrix = confusion_matrix(y_test_classes, y_pred_classes)
print("Matrice de confusion :")
print(conf_matrix)

# Rapport de classification
target_names = [str(cls) for cls in lb.classes_]
print("Rapport de classification :")
print(classification_report(y_test_classes, y_pred_classes, target_names=target_names))

Matrice de confusion :
[[18  0  0]
 [ 0 12  1]
 [ 0  0 31]]
Rapport de classification :
              precision    recall  f1-score   support

     Healthy       1.00      1.00      1.00        18
    septoria       1.00      0.92      0.96        13
 stripe_rust       0.97      1.00      0.98        31

    accuracy                           0.98        62
   macro avg       0.99      0.97      0.98        62
weighted avg       0.98      0.98      0.98        62



### _____________________________________________________ DenseNet169 _____________________________________

In [31]:
from tensorflow.keras.applications import DenseNet169

# Construction du modèle MobileNetV2
DenseNet169_model = DenseNet169(input_shape=(224, 224, 3), weights="imagenet", include_top=False)
DenseNet169_model.trainable = False

model = models.Sequential([
    DenseNet169_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dense(labels.shape[1], activation='softmax')  # Nombre de classes dynamique
])

# Compilation du modèle
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

# Vérification des types de données
print("X_augmented_train dtype:", X_augmented_train.dtype)
print("y_augmented_train dtype:", y_augmented_train.dtype)


X_augmented_train dtype: float64
y_augmented_train dtype: float64


In [32]:
# Entraînement du modèle avec les données augmentées
model.fit(X_augmented_train, y_augmented_train, epochs=20, validation_data=(X_valid, y_valid))


Epoch 1/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 141s 4s/step - accuracy: 0.7397 - loss: 0.6108 - val_accuracy: 0.8361 - val_loss: 0.3279
Epoch 2/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 62s 3s/step - accuracy: 0.9734 - loss: 0.1089 - val_accuracy: 0.9344 - val_loss: 0.2357
Epoch 3/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 63s 3s/step - accuracy: 1.0000 - loss: 0.0414 - val_accuracy: 0.9344 - val_loss: 0.1781
Epoch 4/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 62s 3s/step - accuracy: 1.0000 - loss: 0.0264 - val_accuracy: 0.9508 - val_loss: 0.1741
Epoch 5/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 61s 3s/step - accuracy: 1.0000 - loss: 0.0142 - val_accuracy: 0.9508 - val_loss: 0.1990
Epoch 6/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 60s 3s/step - accuracy: 1.0000 - loss: 0.0097 - val_accuracy: 0.9344 - val_loss: 0.2029
Epoch 7/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 61s 3s/step - accuracy: 1.0000 - loss: 0.0084 - val_accuracy: 0.9344 - val_loss: 0.2090
Epoch 8/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 63s 3s/step - accuracy: 1.0000 - loss: 0.0064 - val_accuracy: 0.9508 - val_loss

In [33]:
# Évaluation sur l'ensemble test
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Précision sur l'ensemble test : {accuracy:.2f}")


2/2 ━━━━━━━━━━━━━━━━━━━━ 7s 2s/step - accuracy: 0.9577 - loss: 0.1083
Précision sur l'ensemble test : 0.97


In [34]:
# Prédictions
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_test_classes = np.argmax(y_test, axis=1)


2/2 ━━━━━━━━━━━━━━━━━━━━ 40s 19s/step


In [35]:
from sklearn.metrics import confusion_matrix, classification_report

# Matrice de confusion et rapport de classification
conf_matrix = confusion_matrix(y_test_classes, y_pred_classes)
print("Matrice de confusion :")
print(conf_matrix)

# Rapport de classification
target_names = [str(cls) for cls in lb.classes_]
print("Rapport de classification :")
print(classification_report(y_test_classes, y_pred_classes, target_names=target_names))


Matrice de confusion :
[[16  0  2]
 [ 0 13  0]
 [ 0  0 31]]
Rapport de classification :
              precision    recall  f1-score   support

     Healthy       1.00      0.89      0.94        18
    septoria       1.00      1.00      1.00        13
 stripe_rust       0.94      1.00      0.97        31

    accuracy                           0.97        62
   macro avg       0.98      0.96      0.97        62
weighted avg       0.97      0.97      0.97        62

